# Simulation impot concubains avec enfants biologiques.

In [1]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

2019-04-26 18:09:45


In [2]:
year = 2013

In [3]:
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt

In [5]:
from __future__ import division
import pandas as pd ; pd.set_option("display.max_columns",200)
import numpy as np
import pickle
import statsmodels.api as sm
import sys

In [6]:
import sys
sys.path.append(".\optimisation_concubains")


In [7]:
from __future__ import division
import pandas as pd ; pd.set_option("display.max_columns",200)
import numpy as np
import pickle
import statsmodels.api as sm
import sys

In [8]:
store_path = '../Data/hdf/edp_2015_final.h5'

In [9]:
#Store = pd.HDFStore(store_path)

In [10]:
def load_fisc_i_by_year(year = None):
    df_fisci = pd.read_hdf(store_path, 'FISC_INDIVIDU_{}'.format(year))
    return df_fisci
def load_fisc_r_by_year(year = None):
    df_fiscr = pd.read_hdf(store_path, 'FISC_REVENU_{}'.format(year))
    return df_fiscr
def load_fisc_l_by_year(year = None):
    df_fiscr = pd.read_hdf(store_path, 'FISC_LOGEMENT_{}'.format(year))
    return df_fiscr

def load_df_fiscrevdet_by_year(year = None):
    df_fiscrevdet = pd.read_hdf(store_path, 'FISC_REVDET_{}'.format(year))
    return df_fiscrevdet
    

In [11]:
diploma_labels = {1:"No_school", 2 : "primary_middle_school",11:"CEP", 12:"BEPC", 13:"CAP", 14:"BEP", 15:"BAC", 16:"Bac_pro",17:"Bac_plus_3",18:"Master_PhD" }
diploma_labels_homme = {1:"No_school_man", 2 : "primary_middle_school_man",11:"CEP_man", 12:"BEPC_man", 13:"CAP_man", 14:"BEP_man", 15:"BAC_man", 16:"Bac_pro_man",17:"Bac_plus_3_man",18:"Master_PhD_man" }
diploma_labels_femme = {1:"No_school_wo", 2 : "primary_middle_school_wo",11:"CEP_wo", 12:"BEPC_wo", 13:"CAP_wo", 14:"BEP_wo", 15:"BAC_wo", 16:"Bac_pro_wo",17:"Bac_plus_3_wo",18:"Master_PhD_wo" }


def add_diplomas_dummies(df_temp):
    df_temp.loc[df_temp.Diplome_femme == 3, "Diplome_femme"] = 2
    df_temp.loc[df_temp.Diplome_homme == 3, "Diplome_homme"] = 2
    df_temp.loc[df_temp.Diplome_femme == 19, "Diplome_femme"] = 18
    df_temp.loc[df_temp.Diplome_homme == 3, "Diplome_homme"] = 18
    diplome_homme = pd.get_dummies(df_temp.Diplome_homme)
    diplome_femme = pd.get_dummies(df_temp.Diplome_femme)
    diplome_homme = diplome_homme.rename(columns=diploma_labels_homme)
    diplome_femme = diplome_femme.rename(columns=diploma_labels_femme)
    df_temp = pd.concat([df_temp, diplome_homme], axis = 1)
    df_temp = pd.concat([df_temp, diplome_femme], axis = 1)
    
    df_temp["No_education_femme"] = df_temp.Diplome_femme.isin([1,2,11,12])
    df_temp["No_education_homme"] = df_temp.Diplome_homme.isin([1,2,11,12])
    
    df_temp["Vocational_education_femme"] = df_temp.Diplome_femme.isin([13,14,15,16])
    df_temp["Vocational_education_homme"] = df_temp.Diplome_homme.isin([13,14,15,16])
    
    df_temp["Licence_education_femme"] = df_temp.Diplome_femme.isin([17])
    df_temp["Licence_education_homme"] = df_temp.Diplome_homme.isin([17])

    df_temp["Master_PhD_education_femme"] = df_temp.Diplome_femme.isin([18])
    df_temp["Master_PhD_education_homme"] = df_temp.Diplome_homme.isin([18])
    
    
    
    return df_temp

In [12]:
hdf_path = ("./Generated_hdf/optimizers.h5")

In [13]:
biologic_path = (u"./Pickle/biologic/biologic_households{}.p".format(2013))
biologic_2013 = pickle.load( 
        open(biologic_path, 'rb'))

In [14]:
biologic_path = (u"./Pickle/biologic/biologic_households{}.p".format(2014))
biologic_2014 = pickle.load( 
        open(biologic_path, 'rb'))

In [15]:
df = pd.read_hdf(hdf_path,  "optimizers_{}".format(year)).reset_index()
df_2013 = df[df.ID_FISC_LOG_DIFF.isin(biologic_2013)]

In [16]:
df_homme_femme_ear= pd.read_hdf(
    "./Generated_hdf/FSCI_EAR_BIO.h5",  
    "ear_groupby_id_fisc_log_diff_homme_femme").reset_index()

In [17]:
df = pd.read_hdf(hdf_path,  "optimizers_{}".format(2014)).reset_index()
df_2014 = df[df.ID_FISC_LOG_DIFF.isin(biologic_2014)]

In [18]:
pd.HDFStore(hdf_path)

<class 'pandas.io.pytables.HDFStore'>
File path: ./Generated_hdf/optimizers.h5
/optimizers_2013                                              frame        (shape->[173411,120])
/optimizers_2014                                              frame        (shape->[175339,107])
/optimizers_counterfactual_income_2013_on_tax_2014            frame        (shape->[175339,14]) 
/optimizers_counterfactual_income_2014_on_tax_2013            frame        (shape->[173411,14]) 

In [19]:
df_counterfactual_1 = pd.read_hdf(hdf_path,"optimizers_counterfactual_income_2013_on_tax_2014"  )
df_counterfactual_2 = pd.read_hdf(hdf_path,"optimizers_counterfactual_income_2014_on_tax_2013"  )

In [20]:
df_counterfactual = pd.merge(df_counterfactual_1.reset_index(), df_counterfactual_2.reset_index(),
                             on = "ID_FISC_LOG_DIFF")

In [21]:
df_counterfactual.drop_duplicates(subset = "ID_FISC_LOG_DIFF", inplace = True)

In [22]:
df_counterfactual.ID_FISC_LOG_DIFF.nunique()

122291

In [23]:
df_2013.shape

(52328, 122)

In [24]:
df_2013 = pd.merge(df_2013, df_counterfactual, on = "ID_FISC_LOG_DIFF", how = "left")

In [25]:
df_2013.shape

(52328, 150)

## Pour créer le groupe ayant une unique allocation optimale

In [26]:
%%time
i = 0
two_smallest_difference = np.array([])
for df_temp in [df_2013,df_2014]:
    for series in df_temp[["Irpp_decote_allocation_{}".format(alloc_nb) for alloc_nb in range(1,9)]].iterrows():
        two_smallest_allocations =  series[1].nsmallest(2)
        i+=1
        two_smallest_difference = np.append(two_smallest_difference,  two_smallest_allocations[0] - two_smallest_allocations[1]) 
    df_temp["Two_smallest_allocation_difference"] = -pd.Series(two_smallest_difference)


Wall time: 43.4 s


C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\IPython\kernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### On merge avec les infos des EAR

In [27]:
df_2013_save = df_2013
df_2014_save = df_2014

In [28]:
df_2013 = df_2013_save
df_2014 = df_2014_save

In [29]:
df_2013 = pd.merge(df_2013, df_homme_femme_ear, on= "ID_FISC_LOG_DIFF", how = 'left')
df_2014 = pd.merge(df_2014, df_homme_femme_ear, on= "ID_FISC_LOG_DIFF", how = 'left')

### Add change in matrimonial status

In [30]:
path = (u"./Pickle/optimize/")
mariage_id_2013_2014 = pickle.load(open(path+"marriage_en_2013_2014.p", 'rb'))
pacs_id_2013_2014 = pickle.load(open(path+"pacs_en_2013_2014.p", 'rb'))
separation_id_2013_2014 = pickle.load(open(path+"separation_en_2013_2014.p", 'rb'))



df_2013['Mariage'] = df_2013.ID_FISC_LOG_DIFF.isin(mariage_id_2013_2014)
df_2013['Pacs'] = df_2013.ID_FISC_LOG_DIFF.isin(pacs_id_2013_2014)
df_2013['Separation'] = df_2013.ID_FISC_LOG_DIFF.isin(separation_id_2013_2014)
df_2013["Mar_Pacs"] = df_2013['Mariage'] | df_2013['Pacs']
df_2013['Do_not_optimize_olivier'] = (df_2013.Fiscal_loss/df_2013.Rni)>0.05
    
    
    

### Add Variables

In [31]:
%%time
for df_temp in [df_2013, df_2014]:
    i = 0
    two_smallest_difference = np.array([])
    for series in df_temp[["Irpp_decote_allocation_{}".format(alloc_nb) for alloc_nb in range(1,9)]].iterrows():
        two_smallest_allocations =  series[1].nsmallest(2)
        i+=1
        two_smallest_difference = np.append(two_smallest_difference,  two_smallest_allocations[0] - two_smallest_allocations[1]) 
    df_temp["Two_smallest_allocation_difference"] = two_smallest_difference

Wall time: 39.1 s


In [32]:
df_2013["No_decote"] = (df_2013.Irpp_decote==df_2013.Irpp_ac_plaf_qf)
df_2014["No_decote"] = (df_2014.Irpp_decote==df_2014.Irpp_ac_plaf_qf)

In [33]:
%%capture
for df_temp in [df_2013, df_2014]:
    df_temp["deux_enfants"] = df_temp.Nb_enfants == 2
    df_temp["trois_enfants_et_plus"] = df_temp.Nb_enfants >= 3 
    df_temp["trois_enfants"] = df_temp.Nb_enfants == 3
    df_temp["quatre_enfants_et_plus"] = df_temp.Nb_enfants >= 4 
    df_temp["Age_moyen"] = (df_temp.Age_mere + df_temp.Age_pere)/2
    df_temp["Age_moyen_parents"] = (df_temp.Age_mere + df_temp.Age_pere)/2
    df_temp["Age_moyen_squared"] = df_temp.Age_moyen**2
    df_temp["Ecart_Age_en_valeur_absolue"] = np.abs(df_temp.Age_pere - df_temp.Age_mere)
    df_temp["Prct_rev_perte_fiscale"] = df_temp.Fiscal_loss/df_temp.Rni
    df_temp["Diff_in_incompe_mere_pere"] = np.abs(df_temp.Rni_pere - df_temp.Rni_mere)

In [34]:
df_2013.shape

(52336, 187)

### Age enfants

In [35]:
df_fisci = load_fisc_i_by_year(2013)

In [36]:
df_fisci_select = df_fisci[df_fisci.TYPE_FISC.isin(["A", "B", "C","D", "E", "F","G", "H", "I"])] #Garde que les enfants

In [37]:
df_fisci_select = df_fisci_select.sort(["ID_FISC_LOG_DIFF", "ANAIS"])

In [38]:
grpby = pd.DataFrame()

In [39]:
grpby["Age_elder_child"] = 2013- df_fisci_select[["ID_FISC_LOG_DIFF", "ANAIS"]].groupby("ID_FISC_LOG_DIFF").ANAIS.first()
grpby["Age_younger_child"] = 2013 - df_fisci_select[["ID_FISC_LOG_DIFF", "ANAIS"]].groupby("ID_FISC_LOG_DIFF").ANAIS.last()
grpby["Age_moyen_children"] = 2013 - df_fisci_select[["ID_FISC_LOG_DIFF", "ANAIS"]].groupby("ID_FISC_LOG_DIFF").ANAIS.mean()

In [40]:
#grpby.head()

In [41]:
df_2013 = pd.merge(df_2013, grpby.reset_index(), on = "ID_FISC_LOG_DIFF", how = "left")
df_2014 = pd.merge(df_2014, grpby.reset_index(), on = "ID_FISC_LOG_DIFF", how = "left")

### Add years

In [42]:
df_2013.shape

(52336, 190)

In [43]:
df_2013["Annee"] = 2013
df_2013["year_2013"] =(df_2013.Annee == 2013)
df_2014["Annee"] = 2014
df_2014["year_2014"] =(df_2014.Annee == 2014)

In [44]:
df_2013.ID_FISC_LOG_DIFF;

### Add diplomas 

In [45]:
df_2013 = add_diplomas_dummies(df_2013)
df_2014 = add_diplomas_dummies(df_2014)

In [46]:
diploma_4_categories = ["No_education_femme", "Vocational_education_femme","Licence_education_femme", "Master_PhD_education_femme",
                       "No_education_homme", "Vocational_education_homme","Licence_education_homme", "Master_PhD_education_homme"]
diploma_4_categories_to_regress =  ["Vocational_education_femme","Licence_education_femme", "Master_PhD_education_femme",
                       "Vocational_education_homme","Licence_education_homme", "Master_PhD_education_homme"]

In [47]:
for df_temp in [df_2013, df_2014]:
    df_temp["Same_diploma"] = (df_temp["No_education_femme"]*df_temp["No_education_homme"]+
                            df_temp["Vocational_education_femme"]*df_temp["Vocational_education_homme"]+
                            df_temp["Licence_education_femme"]*df_temp["Licence_education_homme"]+
                            df_temp["Master_PhD_education_femme"]*df_temp["Master_PhD_education_homme"])


    df_temp["Same_diploma_No_educ"] = (df_temp["No_education_femme"]*df_temp["No_education_homme"])
    df_temp["Same_diploma_Vocational"] = (df_temp["Vocational_education_femme"]*df_temp["Vocational_education_homme"])
    df_temp["Same_diploma_licence"] =  (df_temp["Licence_education_femme"]*df_temp["Licence_education_homme"])
    df_temp["Same_diploma_Master"] = (df_temp["Master_PhD_education_femme"]*df_temp["Master_PhD_education_homme"])

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\pandas\computation\expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))
C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\pandas\computation\expressions.py:190: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


In [48]:
print df_2013.shape
print df_2013[df_2013[diploma_4_categories].sum(axis = 1) == 2].shape

(52336, 226)
(33044, 226)


In [49]:
var_revenus_fiscr = ['ZTSAM',
       u'ZSALM', u'ZCHOM', u'ZPERM', u'ZRETM', u'ZRSTM', u'ZALRM', u'ZRTOM',
       u'ZRAGM', u'ZRICM', u'ZRNCM', u'ZFONM', u'ZVAMM0', u'ZVAMM', u'ZVALM0',
       u'ZVALM', u'ZRACM', u'ZETRM', u'ZALVM']

var_revenus_fiscrevdet = ['YSALI','YCHOI','YRAGI','YBICI','YBNCI','YRSTI','YALRI']

In [50]:
df_2013_can_optimize = df_2013[(df_2013.result_min_allocation < df_2013.result_max_allocation )]
df_2014_can_optimize = df_2014[(df_2014.result_min_allocation < df_2014.result_max_allocation )]
df_no_decote_2013 = df_2013[(df_2013.Irpp_decote==df_2013.Irpp_ac_plaf_qf)]
df_no_decote_2014 = df_2014[(df_2014.Irpp_decote==df_2014.Irpp_ac_plaf_qf)]
df_no_decote_can_optimize_2013 = df_2013[(df_2013.Irpp_decote==df_2013.Irpp_ac_plaf_qf) &  (df_2013.result_min_allocation < df_2013.result_max_allocation )]
df_no_decote_can_optimize_2014 = df_2014[(df_2014.Irpp_decote==df_2014.Irpp_ac_plaf_qf)& (df_2014.result_min_allocation < df_2014.result_max_allocation )]

In [51]:
df2013_A = df_2013
df2013_B = df_2013_can_optimize
df2013_C = df_no_decote_can_optimize_2013
df2013_D = df2013_B[df2013_B.Two_smallest_allocation_difference<0]
df2014_A = df_2014
df2014_B = df_2014_can_optimize
df2014_C = df_no_decote_can_optimize_2014
df2014_D = df2014_B[df2014_B.Two_smallest_allocation_difference<0]

In [52]:
print(df2013_B.query("Nb_enfants == 1").shape,df2013_D.query("Nb_enfants == 1").shape)

((24548, 226), (24548, 226))


In [53]:
#Test that for 1 child families the number of "can optimize household" is the same as "unique optimal allocation"
assert (df2013_B.query("Nb_enfants == 1").shape == df2013_D.query("Nb_enfants == 1").shape)

In [54]:
df_A = pd.merge(df2013_A, df2014_A, on="ID_FISC_LOG_DIFF", suffixes = ("_2013", "_2014"))
df_B = pd.merge(df2013_B, df2014_B, on="ID_FISC_LOG_DIFF", suffixes = ("_2013", "_2014"))
df_C = pd.merge(df2013_C, df2014_C, on="ID_FISC_LOG_DIFF", suffixes = ("_2013", "_2014"))
df_D = pd.merge(df2013_D, df2014_D, on="ID_FISC_LOG_DIFF", suffixes = ("_2013", "_2014"))

In [55]:
df_B.shape

(32272, 405)

In [56]:
irrp_decote_allocation_var_name = ["Irpp_decote_allocation_{}_2013".format(nb_alloc) for nb_alloc in range(1,10)]

In [57]:
for df_group_name, df_temp in {"A":df_A,"B":df_B,"C":df_C}.iteritems():
    print "\n  For group", df_group_name
    df_temp["optimiseurs_2013_2014"]=(~df_temp.Do_not_optimize_2013 & ~df_temp.Do_not_optimize_2014)
    df_temp["non_opt_2013_opt_2014"]=(df_temp.Do_not_optimize_2013 & ~df_temp.Do_not_optimize_2014)
    df_temp["opt_2013_non_opt_2014"]=(~df_temp.Do_not_optimize_2013 & df_temp.Do_not_optimize_2014)
    df_temp["non_opt_2013_2014"]=(df_temp.Do_not_optimize_2013 & df_temp.Do_not_optimize_2014)



  For group A

  For group C

  For group B


### Same optimal allocation

In [58]:
for sub_group in ["optimiseurs_2013_2014","non_opt_2013_opt_2014","opt_2013_non_opt_2014","non_opt_2013_2014"]:
    #print sub_group
    for df_group_name, df_temp in {"A":df_A.query("{}".format(sub_group)),
                                   "B":df_B.query("{}".format(sub_group)),
                                   "C":df_C.query("{}".format(sub_group))}.iteritems():
        #print "For group", df_group_name, ", sample size = ", df_temp.shape[0]
        melt_2013 = pd.melt(df_temp[["ID_FISC_LOG_DIFF"]+ ["Irpp_decote_allocation_{}_2013".format(nb_alloc) for nb_alloc in range(1,10)]]
                    , id_vars = "ID_FISC_LOG_DIFF")
        idx_2013 = melt_2013.groupby("ID_FISC_LOG_DIFF").value.transform(min) == melt_2013.value
        melt_2014 = pd.melt(df_temp[["ID_FISC_LOG_DIFF"]+ ["Irpp_decote_allocation_{}_2014".format(nb_alloc) for nb_alloc in range(1,10)]]
                    , id_vars = "ID_FISC_LOG_DIFF")
        idx_2014 = melt_2014.groupby("ID_FISC_LOG_DIFF").value.transform(min) == melt_2014.value
        variable_2014 = melt_2014[idx_2014].sort("ID_FISC_LOG_DIFF")[["ID_FISC_LOG_DIFF","variable"]]
        variable_2013 = melt_2013[idx_2013].sort("ID_FISC_LOG_DIFF")[["ID_FISC_LOG_DIFF","variable"]]
        df_temp = pd.merge(variable_2014, df_temp, on="ID_FISC_LOG_DIFF")
        df_temp = pd.merge(variable_2013, df_temp, on="ID_FISC_LOG_DIFF", suffixes = ("_2013", "_2014"))
        df_temp["Same_optimal_allocation"] = (df_temp.variable_2013.str[:-5] == df_temp.variable_2014.str[:-5]).astype('bool')
        #print df_temp.query("Nb_enfants_2013 == Nb_enfants_2014 ").groupby("ID_FISC_LOG_DIFF").Same_optimal_allocation.sum().value_counts()
        df_temp.query("Nb_enfants_2013 == Nb_enfants_2014 ").groupby("ID_FISC_LOG_DIFF").Same_optimal_allocation.sum().astype("bool").value_counts(normalize = True)[False]

#### Same optimal allocation (same number of children)

In [59]:
for df_group_name, df_temp in { "A":df_A, "B":df_B,"C":df_C, "D":df_D}.iteritems():
    df_temp.reset_index(inplace=True)
    print "For group", df_group_name, ", sample size = ", df_temp.shape[0]
    melt_2013 = pd.melt(df_temp[["ID_FISC_LOG_DIFF"]+ ["Irpp_decote_allocation_{}_2013".format(nb_alloc) for nb_alloc in range(1,10)]]
                , id_vars = "ID_FISC_LOG_DIFF")
    idx_2013 = melt_2013.groupby("ID_FISC_LOG_DIFF").value.transform(min) == melt_2013.value
    melt_2014 = pd.melt(df_temp[["ID_FISC_LOG_DIFF"]+ ["Irpp_decote_allocation_{}_2014".format(nb_alloc) for nb_alloc in range(1,10)]]
                , id_vars = "ID_FISC_LOG_DIFF")
    idx_2014 = melt_2014.groupby("ID_FISC_LOG_DIFF").value.transform(min) == melt_2014.value
    variable_2014 = melt_2014[idx_2014].sort("ID_FISC_LOG_DIFF")[["ID_FISC_LOG_DIFF","variable"]]
    variable_2013 = melt_2013[idx_2013].sort("ID_FISC_LOG_DIFF")[["ID_FISC_LOG_DIFF","variable"]]
    df_temp_2 = pd.merge(variable_2014, df_temp, on="ID_FISC_LOG_DIFF")
    df_temp_2 = pd.merge(variable_2013, df_temp_2, on="ID_FISC_LOG_DIFF", suffixes = ("_2013", "_2014"))
    df_temp_2["Same_optimal_allocation"] = (df_temp_2.variable_2013.str[:-5] == df_temp_2.variable_2014.str[:-5]).astype('bool')
    #print df_temp.query("Nb_enfants_2013 == Nb_enfants_2014 ").groupby("ID_FISC_LOG_DIFF").Same_optimal_allocation.sum().value_counts()
    print df_temp_2.query("Nb_enfants_2013 == Nb_enfants_2014 ").groupby("ID_FISC_LOG_DIFF").Same_optimal_allocation.sum(
                    ).astype("bool").value_counts(normalize = True)[False]
    ##Warning!!! attention fait à l'arrache, on est censé avoir autent d'obs dans df_temp que dans df_temp_2, chercher d'ou viennent les drop
    df_temp.set_index("ID_FISC_LOG_DIFF", inplace = True)
    df_temp["Same_optimal_allocation_same_family"] =  df_temp_2.query("Nb_enfants_2013 == Nb_enfants_2014 ").groupby("ID_FISC_LOG_DIFF").Same_optimal_allocation.sum()
    df_temp.Same_optimal_allocation_same_family.fillna(0, inplace = True) ; df_temp.reset_index("ID_FISC_LOG_DIFF", inplace = True)


For group A , sample size =  38249
0.144881290436
For group C , sample size =  4838
0.117737338303
For group B , sample size =  32272
0.170708150805
For group D , sample size =  27656
0.189263646739


In [60]:
for df_group_name, df_temp in {"A":df_A,"B":df_B,"C":df_C, "D":df_D}.iteritems():
    print df_group_name
    ##
    ## Rajouté après allocations optimales (flemme de bien nettoyer plus haut)
    ##
    
    melt_2013 = pd.melt(df_temp[["ID_FISC_LOG_DIFF"]+ ["Irpp_decote_allocation_{}_2013".format(nb_alloc) for nb_alloc in range(1,10)]]
                , id_vars = "ID_FISC_LOG_DIFF")
    idx_2013 = melt_2013.groupby("ID_FISC_LOG_DIFF").value.transform(min) == melt_2013.value
    melt_2014 = pd.melt(df_temp[["ID_FISC_LOG_DIFF"]+ ["Irpp_decote_allocation_{}_2014".format(nb_alloc) for nb_alloc in range(1,10)]]
                , id_vars = "ID_FISC_LOG_DIFF")
    idx_2014 = melt_2014.groupby("ID_FISC_LOG_DIFF").value.transform(min) == melt_2014.value
    variable_2014 = melt_2014[idx_2014].sort("ID_FISC_LOG_DIFF")[["ID_FISC_LOG_DIFF","variable"]]
    variable_2013 = melt_2013[idx_2013].sort("ID_FISC_LOG_DIFF")[["ID_FISC_LOG_DIFF","variable"]]
    
    
    
    variable_2013["Allocations_optimales_2013"] = variable_2013.sort("ID_FISC_LOG_DIFF").variable.str[12:24]
    variable_2014["Allocations_optimales_2014"] = variable_2014.sort("ID_FISC_LOG_DIFF").variable.str[12:24]
    #df_temp["Test_allocaton_optimale"] = variable_2013.groupby("ID_FISC_LOG_DIFF", as_index = True).sum()[Allocations_optimales_2013]
    df_temp.set_index('ID_FISC_LOG_DIFF', inplace = True)
    df_temp["Optimal_allocations_2013"] = variable_2013.groupby("ID_FISC_LOG_DIFF", as_index = True).sum()["Allocations_optimales_2013"] #TODO mettre ça dans préparation générale
    df_temp["Optimal_allocations_2014"] = variable_2014.groupby("ID_FISC_LOG_DIFF", as_index = True).sum()["Allocations_optimales_2014"]
    df_temp['Chosen_allocation_2013'] = df_temp.Nb_enfants_2013+1 - df_temp.Allocation_mere_2013
    df_temp['Chosen_allocation_2014'] = df_temp.Nb_enfants_2014+1 - df_temp.Allocation_mere_2014
    
    df_temp["Number_of_optimal_allocations_2013"] = df_temp.Optimal_allocations_2013.str.len()/12
    df_temp["Number_of_optimal_allocations_2014"] = df_temp.Optimal_allocations_2014.str.len()/12
    
    df_temp.reset_index(inplace = True)


A
C
B
D


In [61]:
## Pour l'année 2013 
for df_group_name, df_temp in {"A":df2013_A,"B":df2013_B,"C":df2013_C, "D":df2013_D}.iteritems():
    print df_group_name
    ##
    ## Rajouté après allocations optimales (flemme de bien nettoyer plus haut)
    ##
    
    melt_2013 = pd.melt(df_temp[["ID_FISC_LOG_DIFF"]+ ["Irpp_decote_allocation_{}".format(nb_alloc) for nb_alloc in range(1,10)]]
                , id_vars = "ID_FISC_LOG_DIFF")
    idx_2013 = melt_2013.groupby("ID_FISC_LOG_DIFF").value.transform(min) == melt_2013.value
    variable_2013 = melt_2013[idx_2013].sort("ID_FISC_LOG_DIFF")[["ID_FISC_LOG_DIFF","variable"]]
    
    
    
    variable_2013["Allocations_optimales"] = variable_2013.sort("ID_FISC_LOG_DIFF").variable.str[12:24]
    #df_temp["Test_allocaton_optimale"] = variable_2013.groupby("ID_FISC_LOG_DIFF", as_index = True).sum()[Allocations_optimales_2013]
    df_temp.set_index('ID_FISC_LOG_DIFF', inplace = True)
    df_temp["Optimal_allocations"] = variable_2013.groupby("ID_FISC_LOG_DIFF", as_index = True).sum()["Allocations_optimales"] #TODO mettre ça dans préparation générale
    df_temp['Chosen_allocation'] = df_temp.Nb_enfants+1 - df_temp.Allocation_mere
    
    df_temp["Number_of_optimal_allocations"] = df_temp.Optimal_allocations.str.len()/12
    
    df_temp.reset_index(inplace = True)


A
C

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\IPython\kernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\IPython\kernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



B
D


C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\IPython\kernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [62]:
# melt_2013 = pd.melt(df_temp[["ID_FISC_LOG_DIFF"]+ ["Irpp_decote_allocation_{}_2013".format(nb_alloc) for nb_alloc in range(1,10)]]
#                 , id_vars = "ID_FISC_LOG_DIFF")

In [63]:
melt_2014["min_alloc"] = melt_2014.groupby("ID_FISC_LOG_DIFF").value.transform(min)

In [64]:
#melt_2014.sort("ID_FISC_LOG_DIFF").tail()

In [65]:
melt_2014["Alloc_is_optimal"] = (melt_2014.value == melt_2014.min_alloc).astype("int")

In [66]:
melt_2014.groupby("ID_FISC_LOG_DIFF").Alloc_is_optimal.sum().value_counts()

1    27648
4        2
dtype: int64

In [67]:
#TODO fonctionne plus, voir data_preparation_copy_1 pour avoir le bon output.
(df_A.Optimal_allocations_2013.str.len()/12).value_counts()

1     32081
2      3845
3      1761
4       342
5       171
6        22
7        21
36        4
12        2
dtype: int64

Les allocations sont classées comme suit : l'allocation 1 correspond à l'allocation ou la même prend tout les enfants, et la dernière allocation correspond à l'allocation ou le père prend tout les enfants.

In [68]:
#df_B[["Test_to_trash","Chosen_allocation", "Allocation_mere_2013", "Allocation_pere_2013", "Do_not_optimize_2013"]+['Irpp_decote_allocation_{}_2013'.format(aloc) for aloc in range(1,6)]]

In [69]:
for df_temp in [df_A,df_B,df_C, df_D]:
    same_family_same_allocation = ((df_temp.Nb_enfants_2013 == df_temp.Nb_enfants_2014)&
                              (df_temp.Allocation_mere_2013 == df_temp.Allocation_mere_2014))
    same_family_different_allocation = ((df_temp.Nb_enfants_2013 == df_temp.Nb_enfants_2014)&
                              (df_temp.Allocation_mere_2013 != df_temp.Allocation_mere_2014))
    df_temp["Same_family_did_not_change_allocation"] = same_family_same_allocation.astype('int')
    df_temp["Same_family_did_change_allocation"] = same_family_different_allocation.astype('int')
    
    
    df_temp["Family_change"] = df_temp.Nb_enfants_2013 != df_temp.Nb_enfants_2014
    df_temp["Family_child_gain"] = df_temp.Nb_enfants_2013 < df_temp.Nb_enfants_2014
    df_temp["Family_child_loss"] = df_temp.Nb_enfants_2013 > df_temp.Nb_enfants_2014

In [70]:
df_concat_A = pd.concat([df2013_A, df2014_A])
df_concat_B = pd.concat([df2013_B, df2014_B])
df_concat_C = pd.concat([df2013_C, df2014_C])
df_concat_D = pd.concat([df2013_D, df2014_D])

In [71]:
%%capture
for df_temp in [df_concat_A, df_concat_B, df_concat_C, df_concat_D]:
    df_temp["year_2013"].fillna(False, inplace=True)
    df_temp["year_2014"].fillna(False, inplace=True)

In [72]:
from IPython.display import display

In [73]:
#df_concat_A.Ecart_Age_en_valeur_absolue.value_counts(dropna=False)

In [74]:
df_B.shape

(32272, 422)

### Keep households with diplomas only


In [75]:
diploma_4_categories_2013 = [a + "_2013" for a in diploma_4_categories]

In [76]:
# daframes_list = {"df2013_A":df2013_A,"df2013_B":df2013_B, "df2013_C":df2013_C,"df2013_D":df2013_D, 
#                     "df2014_A":df2014_A,"df2014_B":df2014_B, "df2014_C":df2014_C, "df2014_D":df2014_D,
#                     "df_concat_A":df_concat_A, "df_concat_B":df_concat_B,"df_concat_C":df_concat_C,"df_concat_D":df_concat_D,
#                     "df_A": df_A, "df_B": df_B, "df_C":df_C,"df_D":df_D,}
# for key, df_temp in daframes_list.iteritems():
#         print key, df_temp.shape
#         try:
#             df_temp = df_temp[df_temp[diploma_4_categories].sum(axis = 1) == 2]
#             daframes_list[key] = df_temp
#         except KeyError: #Deal with diplomas variable having the year in the end of the name
#             try: 
#                 df_temp = df_temp[df_temp[diploma_4_categories_2013].sum(axis = 1) == 2]
#                 daframes_list[key] = df_temp
#             except: Boum
                
# for key, df_temp in daframes_list.iteritems():
#         print key, df_temp.shape

In [77]:
daframes_list = {"df2013_A":df2013_A,"df2013_B":df2013_B, "df2013_C":df2013_C,"df2013_D":df2013_D, 
                    "df2014_A":df2014_A,"df2014_B":df2014_B, "df2014_C":df2014_C, "df2014_D":df2014_D,
                    "df_concat_A":df_concat_A, "df_concat_B":df_concat_B,"df_concat_C":df_concat_C,"df_concat_D":df_concat_D,
                    "df_A": df_A, "df_B": df_B, "df_C":df_C,"df_D":df_D,}

### Save data for exploitation

In [78]:
hdf_for_exploitation_path = ("./Generated_hdf/data_to_exploit.h5")

In [79]:
%%time
for key, df_temp in daframes_list.iteritems():
    df_temp.to_hdf(hdf_for_exploitation_path, key)

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\pandas\io\pytables.py:2577: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis0] [items->None]

  warnings.warn(ws, PerformanceWarning)
C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\pandas\io\pytables.py:2577: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_items] [items->None]

  warnings.warn(ws, PerformanceWarning)
C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\pandas\io\pytables.py:2577: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directl

Wall time: 13.9 s


In [80]:
pd.HDFStore(hdf_for_exploitation_path)

<class 'pandas.io.pytables.HDFStore'>
File path: ./Generated_hdf/data_to_exploit.h5
/df2013_A               frame        (shape->[52336,81]) 
/df2013_B               frame        (shape->[46446,81]) 
/df2013_C               frame        (shape->[9049,81])  
/df2013_D               frame        (shape->[43418,81]) 
/df2014_A               frame        (shape->[54313,67]) 
/df2014_B               frame        (shape->[45873,67]) 
/df2014_C               frame        (shape->[13403,67]) 
/df2014_D               frame        (shape->[39853,67]) 
/df_A                   frame        (shape->[38249,422])
/df_B                   frame        (shape->[32272,422])
/df_C                   frame        (shape->[4838,422]) 
/df_D                   frame        (shape->[27656,418])
/df_concat_A            frame        (shape->[106649,82])
/df_concat_B            frame        (shape->[92319,82]) 
/df_concat_C            frame        (shape->[22452,82]) 
/df_concat_D            frame        (shape->[

In [81]:
stop_time = datetime.datetime.now(); print("now: ",stop_time);
execution_time = stop_time - start_time; print("execution_time: ",execution_time)
stop_time = start_time
#clear  memory
%reset -f

('now: ', datetime.datetime(2019, 4, 26, 18, 12, 45, 341000))
('execution_time: ', datetime.timedelta(0, 180, 341000))
